### Exercices SQLite

In [6]:
# 1. Read all the data from bc.csv, how many rows is there ? Columns ? 
import sqlite3
import pandas as pd

data = pd.read_csv('C:/Users/10144/Desktop/NoSQL/datachap1/bc.csv', sep=',')
print(data.shape)

(569, 33)


In [8]:
# 2. Insert the bc.csv in the sqlite format.
conn = sqlite3.connect('C:/Users/10144/Desktop/NoSQL/datachap1/bc.db')
c = conn.cursor()
data.to_sql('bc_data', conn, if_exists='replace', index = False)

569

In [9]:
# 3. In the same db create a new table called tomatch with two variables id_ and rdm_float. 
c.execute('CREATE TABLE tomatch (id_ INTEGER, rdm_float FLOAT)')

In [16]:
# 4.In the table tomatch insert observations where the id_ starts from 50k, ends at 10000000 and is only even numbers. rdm_float is a random float for each observation.
import random
import tqdm
query = """ INSERT INTO tomatch (id_, "rdm_float") VALUES (?,?) """

list_of_insertion = []
for i in tqdm.tqdm(range(50000, 10000001, 2)):
    values = (i,random.random())
    list_of_insertion.append(values)
    if len(list_of_insertion) == 569:
        c.executemany(query, list_of_insertion)
        list_of_insertion = []
conn.commit()
  

100%|██████████| 4975001/4975001 [00:27<00:00, 178570.01it/s]


In [25]:
# 5. Do an Inner join between the two tables and compute the different aggregate functions on a column of your choice.
c.execute(""" SELECT diagnosis,AVG(texture_mean), SUM(texture_mean), COUNT(texture_mean), MIN(texture_mean),MAX(texture_mean)
          FROM bc_data as bc
          INNER JOIN tomatch as t
          ON bc.id = t.id_""")

result = c.fetchall()
print(result)

[(925622, 19.21329218106994, 4668.829999999995, 243, 10.38, 30.62)]


In [22]:
# 6. How many observations have a radius_mean greater than 15 ? From them how many have a texture_mean greater than 20 ?
c.execute(""" SELECT COUNT(*) FROM bc_data
          WHERE radius_mean > 15""")
result1 = c.fetchall()
print(result1)

c.execute("""SELECT COUNT(*) FROM bc_data
          WHERE radius_mean > 15 
          AND texture_mean > 20 """)
result2 = c.fetchall()
print(result2)

[(173,)]
[(106,)]


In [23]:
# 7.For each category in diagnosis do the mean of a variable and round it to get only 2 numbers after the comma. Compute the difference between the mean of both groups

c.execute(""" SELECT diagnosis, ROUND(AVG(radius_mean),2) FROM bc_data
          GROUP BY diagnosis""")
result3 = c.fetchall()
print(result3)


[('B', 12.15), ('M', 17.46)]


In [26]:
# 8.Change the name of the column "diagnosis" to "label"
c.execute(""" ALTER TABLE bc_data RENAME COLUMN diagnosis TO label""")
conn.commit()

In [27]:
# 9.Add a column named "day" of type DATE
c.execute(""" ALTER TABLE bc_data ADD COLUMN day DATE""")
conn.commit()

In [32]:
# 10.  Change the type of the column "area_mean" to int (hint, not possible on sqlite, create a new column)
c.execute(""" UPDATE bc_data SET area_mean_new = CAST(area_mean AS INT)""")
conn.commit()

In [40]:
# 11.create a new table called tomatch2 exactly the same way as tomatch except you increment id_ by 10.
c.execute(""" CREATE TABLE tomatch2 (id_, rdm_float) """)
conn.commit()
query = """ INSERT INTO tomatch2 (id_, "rdm_float") VALUES (?,?) """

list_of_insertion = []
for i in tqdm.tqdm(range(50000, 10000001, 10)):
    values = (i,random.random())
    list_of_insertion.append(values)
    if len(list_of_insertion) == 569:
        c.executemany(query, list_of_insertion)
        list_of_insertion = []
conn.commit()

  0%|          | 0/995001 [00:00<?, ?it/s]

100%|██████████| 995001/995001 [00:06<00:00, 144494.73it/s]


In [45]:
''' 12. Do an inner join of bc, tomatch and tomatch2 where the radius_mean is greater than 15, 
and the rdm_float is greater than 0.50 in tomatch AND tomatch2. '''

c.execute(""" SELECT bc.id, t.id_, t2.id_ FROM bc_data as bc
          INNER JOIN tomatch as t
          ON bc.id = t.id_
          INNER JOIN tomatch2 as t2
          ON bc.id = t2.id_
          WHERE bc.radius_mean > 15
          AND t.rdm_float > 0.50
          AND t2.rdm_float > 0.50""")
reslut = c.fetchall()
print(result)

[(925622, 19.21329218106994, 4668.829999999995, 243, 10.38, 30.62)]


### Exercices MySQL

In [46]:
import mysql.connector

mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="Lihuiyue12!",
    database="m1_ds2e"
)

c = mydb.cursor()

In [50]:
# 1. How many orders were placed in total?
c.execute(""" SELECT COUNT(*) FROM sample_superstore""")
result = c.fetchall()
print(result)

[(9033,)]


In [52]:
# 2. What is the total sales amount?
c.execute(""" SELECT SUM(Sales) FROM sample_superstore""")
result = c.fetchall()
print(result)

[(8698057.950000014,)]


In [53]:
# 3. What is the average profit per order?
c.execute(""" SELECT AVG(Profit) FROM sample_superstore""")
result = c.fetchall()
print(result)

[(138.4031869077875,)]


In [60]:
# 4. Which customer has placed the most orders?
c.execute("""SELECT `Customer ID`, COUNT(`Order ID`) as order_num 
          FROM sample_superstore
          GROUP BY `Customer ID`
          ORDER BY COUNT(`Order ID`) DESC
          LIMIT 1""") # => syntax: le nom de table contient space doit etre entre ``
result = c.fetchall()
print(result)

[(1193, 26)]


In [66]:
# 5. What is the total profit for each product category?
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="Lihuiyue12!",
    database="m1_ds2e"
)

c = mydb.cursor()

c.execute(""" SELECT `Product Category`, SUM(Profit)
          FROM sample_superstore
          GROUP BY `Product Category`""")
result = c.fetchall()
print(result)


[('Office Supplies', 440488.3892998), ('Technology', 688056.5941278014), ('Furniture', 121651.00391044984)]


In [67]:
# 6. What is the average discount percentage?
c.execute(""" SELECT AVG(Discount)*100 FROM sample_superstore""")
c.fetchall()

[(4.9626923502710785,)]

In [77]:
# 7. Which region has the highest total sales?
c.execute(""" SELECT Region, SUM(Sales) as total_sales
          FROM sample_superstore
          GROUP BY Region
          ORDER BY total_sales DESC
          LIMIT 1 """)
c.fetchall()

[('Central', 2462501.9600000004)]

In [71]:
# 8. How many orders were shipped using each shipping mode?
c.execute(""" SELECT `Ship Mode`, COUNT(`Order ID`) as oreder_num
          FROM sample_superstore
          GROUP BY `Ship Mode`""")
c.fetchall()

[('Regular Air', 6715), ('Express Air', 1052), ('Delivery Truck', 1266)]

In [72]:
# 9. What is the total quantity ordered for each product sub-category?
c.execute(""" SELECT `Product Sub-Category`, SUM(`Quantity ordered new`) as total_order_quantity
          FROM sample_superstore
          GROUP BY `Product Sub-Category`""")
c.fetchall()

[('Labels', Decimal('4794')),
 ('Pens & Art Supplies', Decimal('10752')),
 ('Paper', Decimal('17666')),
 ('Scissors, Rulers and Trimmers', Decimal('1865')),
 ('Telephones and Communication', Decimal('14284')),
 ('Office Machines', Decimal('4674')),
 ('Chairs & Chairmats', Decimal('5769')),
 ('Appliances', Decimal('7040')),
 ('Bookcases', Decimal('2596')),
 ('Tables', Decimal('5037')),
 ('Envelopes', Decimal('3741')),
 ('Office Furnishings', Decimal('11910')),
 ('Rubber Bands', Decimal('2755')),
 ('Binders and Binder Accessories', Decimal('12405')),
 ('Storage & Organization', Decimal('7822')),
 ('Copiers and Fax', Decimal('1177')),
 ('Computer Peripherals', Decimal('10773'))]

In [76]:
# 10. Which product has the highest profit margin? => profit margin = profit/sales
c.execute(""" SELECT `Product Name`, SUM(Profit)/SUM(Sales) as profit_margin
          FROM sample_superstore
          GROUP BY `Product Name`
          ORDER BY profit_margin DESC
          LIMIT 1""")
c.fetchall()

[('Advantus Panel Wall Acrylic Frame', 26.037102272727275)]

In [80]:
# 11. What is the total sales amount for each year?
c.execute(""" SELECT DATE_FORMAT(STR_TO_DATE(`Order Date`, '%m/%d/%Y'), '%Y') as year, SUM(Sales) as total_sales
          FROM sample_superstore
          GROUP BY year""")
# change le format de `order date` qui est le text en format date (STR_TO_DATE()) et puis on extrait l'année
c.fetchall()


[('2012', 2118545.4900000007),
 ('2010', 1864104.4299999992),
 ('2011', 1913396.2600000014),
 ('2013', 2802011.7700000005)]

In [81]:
# 12. Which customer segment has the highest average sales?
c.execute(""" SELECT `Customer Segment`, AVG(Sales) as avg_sales
          FROM sample_superstore
          GROUP BY `Customer Segment`
          ORDER BY avg_sales
          LIMIT 1""")
c.fetchall()

[('Small Business', 913.2110847457618)]

### Exercices PostgreSQL

In [2]:
import psycopg2

# Connect to PostgreSQL database
conn = psycopg2.connect(dbname="new_db", user="postgres", password="Lihuiyue12!", host="localhost",  port="5432" )
c = conn.cursor()

In [3]:
# 1. What is the average age of the customers?
c.execute(""" SELECT AVG(age) FROM bank_full""")
c.fetchall()

[(Decimal('40.9362102143283714'),)]

In [34]:
# 2. How many customers have defaulted on loans?
conn = psycopg2.connect(dbname="new_db", user="postgres", password="Lihuiyue12!", host="localhost",  port="5432" )
c = conn.cursor()
c.execute(""" SELECT COUNT(*) 
          FROM bank_full
          WHERE default_='yes' """)
c.fetchall()

[(815,)]

In [14]:
# 3. What is the maximum balance in the account?
c.execute(""" SELECT MAX(balance) FROM bank_full""")
c.fetchall()

[(Decimal('102127'),)]

In [33]:
# 4. How many customers have a housing loan?
c.execute(""" SELECT COUNT(*) FROM bank_full
          WHERE housing = 'yes' """)
c.fetchall()

[(25130,)]

In [19]:
# 5. What is the education level of the youngest customer?
conn = psycopg2.connect(dbname="new_db", user="postgres", password="Lihuiyue12!", host="localhost",  port="5432" )
c = conn.cursor()
c.execute(""" SELECT education, MIN(age)
          FROM bank_full
          GROUP BY education""")
c.fetchall()

[('secondary', 18), ('primary', 18), ('unknown', 18), ('tertiary', 20)]

In [31]:
# 6. What is the marital status of the oldest customer?
conn = psycopg2.connect(dbname="new_db", user="postgres", password="Lihuiyue12!", host="localhost",  port="5432" )
c = conn.cursor()
c.execute(""" SELECT MAX(age), marital
          FROM bank_full
          GROUP BY marital
          ORDER BY MAX(age) DESC""")
c.fetchall()

[(95, 'divorced'), (95, 'married'), (86, 'single')]

In [43]:
# 7. What is the distribution of the months in which customers were last contacted?
c.execute(""" SELECT COUNT(*) as num_customers, month
          FROM bank_full
          WHERE month IS NOT NULL
          GROUP BY month
          ORDER BY num_customers DESC""")
c.fetchall()

[(13766, 'may'),
 (6895, 'jul'),
 (6247, 'aug'),
 (5341, 'jun'),
 (3970, 'nov'),
 (2932, 'apr'),
 (2649, 'feb'),
 (1403, 'jan'),
 (738, 'oct'),
 (579, 'sep'),
 (477, 'mar'),
 (214, 'dec')]

In [35]:
# 8. What is the average duration of contact for customers who have a personal loan?
c.execute(""" SELECT AVG(duration) as avg_duration
          FROM bank_full
          WHERE loan='yes' """)
c.fetchall()

[(Decimal('250.8453892876863611'),)]

In [36]:
# 9. How many customers were previously contacted, and what was the average duration of their last contact?
c.execute(""" SELECT COUNT(*) as num_customers, AVG(duration) as avg_duration
          FROM bank_full
          WHERE pdays != -1 """)
c.fetchall()

[(8257, Decimal('260.1186871745185903'))]

In [42]:
# 10. What is the distribution of job types among customers?
c.execute(""" SELECT job , COUNT(*) as num_customers
          FROM bank_full
          GROUP BY job
          ORDER BY num_customers DESC""")
c.fetchall()

[('blue-collar', 9732),
 ('management', 9458),
 ('technician', 7597),
 ('admin.', 5171),
 ('services', 4154),
 ('retired', 2264),
 ('self-employed', 1579),
 ('entrepreneur', 1487),
 ('unemployed', 1303),
 ('housemaid', 1240),
 ('student', 938),
 ('unknown', 288)]

In [44]:
# 11. How many customers have been contacted more than once in the current campaign?
c.execute(""" SELECT COUNT(*) as num_customers
          FROM bank_full
          WHERE campaign >1 """)
c.fetchall()

[(27667,)]

In [62]:
# 12. What is the success rate of previous marketing campaigns (poutcome) among customers 
c.execute(""" SELECT poutcome, COUNT(*) as num_customers
          FROM bank_full
          GROUP BY poutcome""")
result1 = c.fetchall()

c.execute(""" SELECT COUNT(*) as success
          FROM bank_full
          WHERE poutcome='success'""")
result2 = c.fetchall()
print(result2[0][0]/result1[0][1])

0.821195652173913
